# Evaluation of Multi-Agent approach using an orchestrator

In [1]:
import sys
import os
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/")
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents")
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/prompt_templates")
from approach_evaluator import ApproachEvaluator
from orchestrator_agent import create_sql_from_natural_text, visualize_graph
visualize_graph()

In [2]:
import pandas as pd

data_dir = os.getenv("DATA_DIRECTORY")
sql_input_file = pd.read_csv("../sample/dev.csv").sample(3)

# Config is just for logging purposes and does not affect the run
run_config = {
    "k_shot_prompting_creator_agent": 2,
    "k_shot_prompting_fixer_agent": 1,
    "k_shot_prompting_feedback_agent": 2,
    "k_shot_prompting_select_relevant_database": 0,
    "k_shot_prompting_select_relevant_tables": 2
}

evaluator = ApproachEvaluator(
    run_config=run_config,
    question_ids=sql_input_file["question_id"],
    approach=create_sql_from_natural_text
)
evaluator.evaluate()

{'query_result': '[(0,)]', 'is_correct': False, 'feedback': "The query did not return any results, indicating there might be an issue with the filtering conditions or field names. The 'OpenDate' field is not recognized, and the 'StatusType' field should be checked for accuracy.", 'updated_query': "SELECT COUNT(*) FROM california_schools WHERE OpenDate BETWEEN '2000-01-01' AND '2005-12-31' AND County = 'Stanislaus' AND StatusType = 'Directly Funded School';"}
{'query_result': '[(0,)]', 'is_correct': False, 'feedback': "The query did not return any results, indicating there might be an issue with the filtering conditions or field names. The 'OpenDate' field is not recognized, and the 'StatusType' field should be checked for accuracy.", 'updated_query': "SELECT COUNT(*) FROM california_schools WHERE OpenDate BETWEEN '2000-01-01' AND '2005-12-31' AND County = 'Stanislaus' AND StatusType = 'Directly Funded School';"}
Orchestrator Agent: An error occurred: 'feedback'
Error with final query: 

{'question_id': [489, 66, 442],
 'predicted_query': ['',
  'SELECT "baseSetSize", "code" FROM "sets" WHERE "block" = \'Masques\' OR "block" = \'Mirage\';',
  'SELECT "Column name: LinkTypeId" FROM postLinks WHERE "Column name: PostId" = \'PKHC\';'],
 'predicted_database': ['', 'card_games', 'codebase_community'],
 'feedbacks': [[],
  [{'query_result': '[]',
    'is_correct': False,
    'feedback': "The query did not return any results because the condition 'block = 'Masques' AND block = 'Mirage'' is contradictory. It's not possible for a single row to have both 'block' values. You should use 'OR' instead of 'AND' to retrieve sets from either 'Masques' or 'Mirage'.",
    'updated_query': "SELECT baseSetSize AS 'Set size', code AS 'Set code' FROM sets WHERE block IN ('Masques', 'Mirage');"},
   {'query_result': "[(350, 'MIR'), (350, 'MMQ'), (143, 'NEM'), (144, 'PCY'), (1, 'PMMQ'), (1, 'PNEM'), (1, 'PPCY'), (167, 'VIS'), (167, 'WTH')]",
    'is_correct': False,
    'feedback': "The query 